In [206]:
from typing import Tuple

import json
import geemap
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import ee
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Polygon

In [207]:
# Initialize the Earth Engine module.
ee.Initialize()

In [208]:

bb: Tuple[int] = (-62.94205,  -9.16795, -61.58455,  -7.77635)

aoi: Polygon = Polygon([ [bb[0], bb[1]], [bb[0], bb[3]], [bb[2], bb[3]], [bb[2],bb[1]]])
aoi_gdf: GeoDataFrame = gpd.GeoDataFrame([], geometry=[aoi],crs=4326)
coords = json.loads(aoi_gdf.geometry.to_json())['features'][0]['geometry']['coordinates']
ee_aoi = ee.Geometry.Polygon(coords)

In [131]:
gpd.read_file('/Users/janpisl/Documents/EPFL/workdir/sample_data/mapbiomas_2021.gpkg')

,IDAlerta,CodeAlerta,Fonte,Bioma,Estado,Municipio,AreaHa,AnoDetec,DataDetec,geometry
0,305239,305161,{SIRADX},AMAZÔNIA,PARÁ,CUMARU DO NORTE,0.78,2021.0,2021-01-01,"MULTIPOLYGON (((-50.96535 -7.85860, -50.96515 ..."
1,305241,305163,{SIRADX},AMAZÔNIA,PARÁ,ALTAMIRA,8.42,2021.0,2021-01-01,"MULTIPOLYGON (((-54.96708 -8.07762, -54.96694 ..."
2,305242,305164,{SIRADX},AMAZÔNIA,PARÁ,ALTAMIRA,7.15,2021.0,2021-01-01,"MULTIPOLYGON (((-54.79985 -7.98596, -54.79968 ..."
3,305244,305166,{SIRADX},AMAZÔNIA,PARÁ,ALTAMIRA,0.39,2021.0,2021-01-01,"MULTIPOLYGON (((-54.74875 -7.66918, -54.74864 ..."
4,305249,305171,{SIRADX},AMAZÔNIA,PARÁ,ALTAMIRA,0.69,2021.0,2021-01-01,"MULTIPOLYGON (((-53.55370 -4.22319, -53.55316 ..."
...,...,...,...,...,...,...,...,...,...,...
53468,505068,503710,{SIRADX},AMAZÔNIA,PARÁ,OURILÂNDIA DO NORTE,3.41,2021.0,2021-11-01,"MULTIPOLYGON (((-51.03317 -7.27412, -51.03315 ..."
53469,505069,503711,{SIRADX},AMAZÔNIA,PARÁ,OURILÂNDIA DO NORTE,0.44,2021.0,2021-11-01,"MULTIPOLYGON (((-50.96833 -7.33190, -50.96819 ..."
53470,505108,503750,{SIRADX},AMAZÔNIA,PARÁ,PORTO DE MOZ,1.71,2021.0,2021-11-01,"MULTIPOLYGON (((-52.06767 -1.70927, -52.06749 ..."
53471,505130,503772,{SIRADX},AMAZÔNIA,PARÁ,PORTO DE MOZ,0.94,2021.0,2021-11-01,"MULTIPOLYGON (((-52.10125 -2.02230, -52.10120 ..."


In [223]:

gdf_radd = gpd.read_file('/Users/janpisl/Documents/EPFL/workdir/sample_data/RADD_Q1_2021.gpkg')
gdf_radd['date'] = pd.to_datetime(gdf_radd.wur_radd_alerts__date)

gdf_glad = gpd.read_file('/Users/janpisl/Documents/EPFL/workdir/sample_data/GLAD_landsat_Q1_2021.gpkg')
gdf_glad['date'] = pd.to_datetime(gdf_glad.umd_glad_landsat_alerts__date)


#gdf_mapb = gpd.read_file('/Users/janpisl/Documents/EPFL/workdir/sample_data/mapbiomas_2021.gpkg')
#gdf_mapb['date'] = pd.to_datetime(gdf_mapb.DataDetec)
#gdf_mapb = gdf_mapb.loc[gdf_mapb.Bioma == 'AMAZÔNIA'].loc[gdf_mapb.date < pd.to_datetime('2021-04-01')]


#https://developers.google.com/earth-engine/apidocs/ee-featurecollection-style
radd_style = {"color": "red", "width": 1, "fillColor": "00000000"}
glad_style = {"color": "red", "width": 1, "fillColor": "00000000"}
mapb_style = {"color": "red", "width": 1, "fillColor": "00000000"}

alerts = [ 
           {'dataset' : gdf_radd,'style' : radd_style},
           {'dataset' : gdf_glad,'style' : glad_style},
           #{'dataset' : gdf_mapb,'style' : mapb_style},

    ]


In [224]:
vis_params = {'bands': ['B2', 'B3', 'B4'], 'min': 0.0, 'max': 3000, 'opacity': 1.0, 'gamma': 1.2}

dates = pd.date_range('2020-11-01', periods=5, freq='m')

series = []

for i in range(len(dates)-1):
    st = dates[i]
    end = dates[i+1]
    start_str = f'{dates[i].year}-{dates[i].month}-{dates[i].day}'
    end_str = f'{dates[i+1].year}-{dates[i+1].month}-{dates[i+1].day}'



    mosaic = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(ee_aoi) \
        .filterDate(ee.Date(start_str), ee.Date(end_str))\
        .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 50)\
        .mosaic()

    mosaic = mosaic.visualize(**vis_params)

    for alert in alerts:
        subset = alert['dataset'].query(f'{st.year}{st.month:02d}{st.day:02d} \
                                          < date < \
                                          {end.year}{end.month:02d}{end.day:02d}')
        if not subset.empty:
            subset_ee = geemap.geopandas_to_ee(subset)
            mosaic = mosaic.blend(subset_ee.style(**alert['style']))


    series.append(mosaic)

coll = ee.ImageCollection(series)
labels = coll.aggregate_array("system:index").getInfo()


In [225]:

Map = geemap.Map(center=[-9.04618993283354, -62.911282916381275], zoom=9)

Map.add_time_slider(coll, labels=labels)
Map

Map(center=[-9.04618993283354, -62.911282916381275], controls=(WidgetControl(options=['position', 'transparent…